In [1]:
from scrape.get_wikipedia_page import WikipediaPage

In [2]:
# Constants
API_URL = "https://en.wikipedia.org/w/api.php"
TITLE = "Compartmental models in epidemiology"
wp = WikipediaPage(API_URL)
# Fetching all content types
# all_contents = wp.get_all_content_types(TITLE)

# print("Page Info:", all_contents["Info"])

In [3]:
res = wp.get_edit_history(TITLE, limit=5)

i 0 5
data dict_keys(['continue', 'warnings', 'query'])
i 1 10
data dict_keys(['continue', 'warnings', 'query'])
i 2 15
data dict_keys(['continue', 'warnings', 'query'])
i 3 20
data dict_keys(['continue', 'warnings', 'query'])
i 4 25
data dict_keys(['continue', 'warnings', 'query'])


In [7]:
len(res[3]['*'])

80880

In [9]:
import re

def convert_wikitext_to_html(wikitext):
    """
    Convert basic wikitext markup to HTML.
    
    Handles bold, italic, and new line conversions.
    """
    # Convert bold and italic (''''text'''' to <b><i>text</i></b>)
    html = re.sub(r"'''''(.*?)'''''", r"<b><i>\1</i></b>", wikitext)
    # Convert bold ('''text''' to <b>text</b>)
    html = re.sub(r"'''(.*?)'''", r"<b>\1</b>", html)
    # Convert italic (''text'' to <i>text</i>)
    html = re.sub(r"''(.*?)''", r"<i>\1</i>", html)
    # Convert new lines to <br>
    html = html.replace("\n", "<br>")

    return html

def save_to_html_file(text, filename):
    """
    Save the provided text as an HTML file.

    :param text: The text to be converted and saved.
    :param filename: The name of the file to save the HTML content in.
    """
    html_content = convert_wikitext_to_html(text)
    html_full_content = f"<html><head><title>Wikipedia Page</title></head><body>{html_content}</body></html>"

    # Write the HTML content to a file
    with open(filename, "w", encoding="utf-8") as file:
        file.write(html_full_content)

# Example usage
wikitext = "'''Bold''' and ''italic'' text.\nNew line here."
filename = "ExampleWikipediaPage.html"
save_to_html_file(wikitext, filename)


In [11]:
filename = "test.html"
save_to_html_file(res[3]['*'], filename)

In [16]:
len(res)

25

In [17]:
# [{"revid": r["revid"], "parentid": r["parentid"], "comment": r["comment"]} for r in res]

In [20]:
import re

def extract_sections(wikitext):
    """
    Extract sections from a Wikipedia article in wikitext format.

    :param wikitext: The wikitext of a Wikipedia article.
    :return: A dictionary of sections with their titles and content.
    """
    sections = {}
    # Regular expression for section titles
    pattern = r"(={2,})([^=]+)\1"

    # Splitting the text based on section titles
    parts = re.split(pattern, wikitext)

    # The first part is usually the introduction or summary
    sections["Summary"] = {'level': 0, "content": parts[0].strip()}

    # Iterate over the rest of the parts to extract sections
    for i in range(1, len(parts), 3):
        level = len(parts[i])  # The number of '=' indicates the level of the section
        title = parts[i + 1].strip()
        content = parts[i + 2].strip()
        sections[title] = {"level": level, "content": content}

    return sections

# Example usage
wikitext_example = "{{short description|Example Article}}\n== Section 1 ==\nContent of section 1\n=== Subsection ===\nContent of subsection."

sections = extract_sections(wikitext_example)
for title, details in sections.items():
    print(f"Title: {title}, Level: {details.get('level', 'Summary')}, Content: {details['content']}\n")
# sections


Title: Summary, Level: 0, Content: {{short description|Example Article}}

Title: Section 1, Level: 2, Content: Content of section 1

Title: Subsection, Level: 3, Content: Content of subsection.



In [24]:

sections = extract_sections(res[0]['*'])
for title, details in sections.items():
    print(f"Title: {title}, Level: {details.get('level', 'Summary')}, Content: {details['content'][0:20]}\n")

Title: Summary, Level: 0, Content: {{short description|

Title: The SIR model, Level: 3, Content: [[File:SIR-Modell.sv

Title: Transition rates, Level: 3, Content: For the full specifi

Title: The SIR model without birth and death, Level: 3, Content: [[File:SIR trajector

Title: The force of infection, Level: 4, Content: Note that in the abo

Title: Exact analytical solutions to the SIR model, Level: 4, Content: In 2014, Harko and c

Title: Numerical solutions to the SIR model with approximations, Level: 4, Content: Numerical solutions 

Title: The SIR model with vital dynamics and constant population, Level: 3, Content: Consider a populatio

Title: Steady-state solutions, Level: 3, Content: The expected duratio

Title: Other compartmental models, Level: 3, Content: There are many modif

Title: Variations on the basic SIR model, Level: 2, Content: 

Title: The SIS model, Level: 3, Content: [[File:SIS System Gr

Title: The SIRD model, Level: 3, Content: [[File:SIRD.svg|thum

Title: The 

In [29]:
import difflib

def compare_texts(text1, text2):
    """
    Generate a diff between two texts.

    :param text1: The first text string.
    :param text2: The second text string.
    :return: The diff between the two texts.
    """
    text1_lines = text1.splitlines()
    text2_lines = text2.splitlines()

    diff = difflib.unified_diff(text1_lines, text2_lines, lineterm='')

    # return diff
    return [str(d) for d in diff]

# Example usage
text_version_1 = "This is the first version of the text."
text_version_2 = "This is the second version of the text."
diff_result = compare_texts(text_version_1, text_version_2)
print(diff_result)


['--- ', '+++ ', '@@ -1 +1 @@', '-This is the first version of the text.', '+This is the second version of the text.']


In [32]:
compare_texts(res[15]['*'], res[14]['*'])


['--- ',
 '+++ ',
 '@@ -9,7 +9,7 @@',
 ' ',
 ' ==The SIR model==',
 ' ',
 '-The \'\'\'SIR model\'\'\'<ref name="Harko">{{Cite journal| vauthors = Harko T, Lobo FS, Mak MK |s2cid=14509477|title=Exact analytical solutions of the Susceptible-Infected-Recovered (SIR) epidemic model and of the SIR model with equal death and birth rates |journal=Applied Mathematics and Computation|language=en|volume=236|pages=184–194|year=2014 |doi=10.1016/j.amc.2014.03.030|bibcode=2014arXiv1403.2160H |arxiv=1403.2160 }}</ref><ref name="Beckley">{{cite journal | vauthors = Beckley R, Weatherspoon C, Alexander M, Chandler M, Johnson A, Bhatt GS |date=2013 |title=Modeling epidemics with differential equations |url=http://www.tnstate.edu/mathematics/mathreu/filesreu/GroupProjectSIR.pdf |journal=Tennessee State University Internal Report |access-date=July 19, 2020}}</ref><ref name="KrogerSchlickeiser">{{Cite journal| vauthors = Kröger M, Schlickeiser R |s2cid=225555567 |title=Analytical solution of the SIR-model